<a href="https://colab.research.google.com/github/Aishah-Manam/PredictingPatientHealthDeterioration/blob/main/Assignment_Data_Mining_Patient_Dataset_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title ⬇️  One‑time Git setup  {display-mode: code}
!git config --global user.name "Aishah-Manam"
!git config --global user.email "aishahmnm@gmail.com"

# Clone your empty GitHub repo to /content
REPO_URL = "https://github.com/Aishah-Manam/PredictingPatientHealthDeterioration"
!git clone $REPO_URL


Cloning into 'PredictingPatientHealthDeterioration'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), done.
Resolving deltas: 100% (1/1), done.


In [8]:
%%bash
# Path of the repo you just cloned (check with !ls /content if unsure)
REPO_DIR="/content/PredictingPatientHealthDeterioration"
echo "Repo dir is $REPO_DIR"


Repo dir is /content/PredictingPatientHealthDeterioration


In [9]:
%%bash
# Recursively find every notebook path under /content
find /content -name '*.ipynb' -print


In [ ]:
import shutil, os, glob

REPO_DIR = "/content/wqd7005-synthetic-ed"

# 1. Save notebook inside repo
nb_path = "/content/WQD7005.ipynb"
shutil.copy(nb_path, os.path.join(REPO_DIR, "notebooks_WQD7005.ipynb"))

# 2. Move dataset, log, figure if they exist
for f in ["synthetic_patients.csv", "prompt_log.csv", "shap_summary.png"]:
    if os.path.exists(f):
        shutil.copy(f, os.path.join(REPO_DIR, f))


In [1]:
# 📦 Install libraries needed (only once per session)
# !pip -q install openai sentence-transformers shap scikit-learn pandas numpy matplotlib seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.5 MB/s eta 0:00:00


Step 1: Install Librariens once every session

Step 2: add openAI API key so colab can talk to gpt, LHS bar

In [ ]:
import os, csv, time, numpy as np, pandas as pd
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
rng = np.random.default_rng(42)

# ----- build patient-day grid -----
n_patients = 500
days = pd.date_range("2025-01-01", periods=31, freq="D")
index = pd.MultiIndex.from_product([range(1, n_patients+1), days],
                                   names=["patient_id","date"])
df = pd.DataFrame(index=index).reset_index()

# ----- numeric vitals -----
def vital_series(base, sd, drift):
    noise = rng.normal(0, sd, len(df))
    day_no = df['date'].dt.dayofyear.values
    return (base + drift * (day_no - day_no.mean())/day_no.std() + noise).round(1)

df["spo2"]        = vital_series(97, 1.5,  0)
df["heart_rate"]  = vital_series(80, 12,  0.2)
df["temperature"] = vital_series(36.8, 0.4, 0.05)
df["sys_bp"]      = vital_series(120, 15, 0.3)
df["dia_bp"]      = vital_series(78, 10, 0.2)
df["weight"]      = vital_series(70, 8,  -0.05)
df["glucose"]     = vital_series(5.5, 1.2, 0.1)

# ----- 10 % missing values -----
for col in ["spo2","heart_rate","temperature","sys_bp","dia_bp","weight","glucose"]:
    mask = rng.random(len(df)) < 0.10
    df.loc[mask, col] = np.nan

# ----- generate short clinical notes using GPT (5 % sample for speed) -----
PROMPT = """
You are a nurse filling daily notes.
Given SPO2={spo2}, HR={hr}, Temp={temp}, BP={sbp}/{dbp}, Weight={wt}, Glucose={glu},
write 1‑2 sentences on patient's general feeling and any symptoms.
If any value is missing, mention 'data not recorded'. Use 'c/o' or 'hx' sometimes.
"""

def make_note(row):
    prompt = PROMPT.format(spo2=row.spo2, hr=row.heart_rate, temp=row.temperature,
                           sbp=row.sys_bp, dbp=row.dia_bp, wt=row.weight, glu=row.glucose)
    resp = client.chat.completions.create(
        model="gpt-3.5-turbo", temperature=0.7,
        messages=[{"role":"system","content":prompt}]
    )
    note = resp.choices[0].message.content.strip()
    # log for transparency
    with open("prompt_log.csv","a") as f:
        f.write(f"{time.time()}\t{prompt}\t{note}\n")
    return note

df["note"] = ""
sample_idx = rng.choice(df.index, size=int(0.05*len(df)), replace=False)
df.loc[sample_idx, "note"] = df.loc[sample_idx].apply(make_note, axis=1)

# save dataset
df.to_csv("synthetic_patient_month.csv", index=False)
df.head()
